# Distributed dialogue

This example initializes an assistant agent and a user agent as separate processes and uses RPC to communicate between them. The full codes can be found in in `examples/distributed/distributed_dialog.py`

To install AgentScope, please follow the steps in [README.md](https://github.com/alibaba/AgentScope/blob/main/README.md#installation).

First, we need to set the model configs properly.

In [1]:
model_configs = [
    {
        "type": "openai",
        "name": "gpt-3.5-turbo",
        "parameters": {
            "api_key": "xxx",
            "organization_id": "xxx",
            "temperature": 0.0
        }
    },
    {
        "type": "openai",
        "name": "gpt-4",
        "parameters": {
            "api_key": "xxx",
            "organization_id": "xxx",
            "temperature": 0.0
        }
    }
]

Then, we need to initialize two agents: an assistant agent and a user agnent.

To facilitate display on jupyter, the agents will be started in a standalone multi-process mode. For a fully distributed version, please refer to `examples/distributed/distributed_dialog.py`.

In [2]:
import agentscope
from agentscope.agents.user_agent import UserAgent
from agentscope.agents.rpc_dialog_agent import RpcDialogAgent

agentscope.init(
    model_configs=model_configs
)

assistant_agent = RpcDialogAgent(
    name="Assistant",
    port=12010,
    sys_prompt="You are a helpful assistant.",
    model="gpt-3.5-turbo",
    use_memory=True,
)
user_agent = UserAgent(
    name="User",
)

2024-01-17 11:31:22.219 | INFO     | agentscope.models:read_model_configs:147 - Load configs for model: gpt-3.5-turbo


Finally, let's write the main process of the dialogue and chat with the assistant.

In [3]:
import time
from loguru import logger

msg = user_agent()
while not msg.content.endswith("exit"):
    msg = assistant_agent(msg)
    logger.chat(msg.update_value())
    time.sleep(0.5)
    msg = user_agent(msg)

2024-01-17 11:32:00.262 | INFO     | agentscope.agents.rpc_agent:launch:501 - Starting rpc server [RpcDialogAgent] at port [12010]...
2024-01-17 11:32:00.792 | INFO     | agentscope.agents.rpc_agent:setup_rcp_agent_server:378 - rpc server [RpcDialogAgent] at port [12010] started successfully
2024-01-17 11:32:04.953 | WARNING  | agentscope.models.model:checking_wrapper:122 - Fail to parsing response: Hello! How can I assist you today?.
 Exception: Expecting value: line 1 column 1 (char 0), 	 Attempt 1 / 1
Assistant: Hello! How can I assist you today?
Assistant: Hello! How can I assist you today?
2024-01-17 11:32:16.205 | WARNING  | agentscope.models.model:checking_wrapper:122 - Fail to parsing response: Sure, here's a joke for you:

Why don't scientists trust atoms?

Because they make up literally everything!.
 Exception: Expecting value: line 1 column 1 (char 0), 	 Attempt 1 / 1
Assistant: Sure, here's a joke for you:

Why don't scientists trust atoms?

Because they make up literally e

2024-01-17 11:32:00.779 | INFO     | agentscope.models:read_model_configs:147 - Load configs for model: gpt-3.5-turbo
